# https://www.youtube.com/watch?v=06TE_U21FK4&list=WL&index=2

In [4]:
%pip install mediapipe
%pip install opencv-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 MB 2.1 MB/s eta 0:00:0000:0100:01
  Using cached absl_py-2.0.0-py3-none-any.whl (130 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 1.5 MB/s eta 0:00:00
  Using cached flatbuffers-23.5.26-py2.py3-none-any.whl (26 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 2.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.6/20.6 MB 2.1 MB/s eta 0:00:0000:0100:01
  Using cached opencv_contrib_python-4.8.1.78-cp37-abi3-macosx_10_16_x86_64.whl (64.4 MB)
  Using cached protobuf-3.20.3-py2.py3-none-any.whl (162 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.0/108.0 kB 1.8 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 kB 2.0 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.2/258.2 kB 2.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 2.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━

In [19]:
import cv2
import mediapipe as mp
import numpy as np

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

11 - left shoulder
12 - right shoulder
13 - left elbow
14 - right elbow
15 - left wrist
16 - right wrist
23 - left hip
24 - right hip
25 - left knee
26 - right knee
27 - left ankle
28 - right ankle
29 - left heel
30 - right heel
31 - left foot index
32 - right foot index

# 1. Make Detections

In [10]:
import cv2

cap = cv2.VideoCapture('InShot_20231226_225432550 .mp4')

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("Can't receive frame (stream end?). Exiting ...")
        break
    cv2.imshow('frame', frame)
    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [36]:
import os
import cv2
import mediapipe as mp
import numpy as np

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

# Get the current working directory
current_dir = os.getcwd()

# Define the video file name
video_file = 'InShot_20230813_115442639_1.mp4'

# Get the full path to the video file
video_path = os.path.join(current_dir, video_file)

# Now you can use this path to open the video file
cap = cv2.VideoCapture(video_path)

## Setup mediapipe instance 
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:   
    while cap.isOpened(): # While camera is open
        ret, frame = cap.read() #frame gives image from camera

        # Detect Stiff and render 
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Make detection
        results = pose.process(image)

        # Recolour back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Extract landmarks
        try: 
            landmarks = results.pose_landmarks.landmark
            height, width, _ = image.shape

            # Get the coordinates of the landmarks
            left_ankle = (int(landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x * width), 
                          int(landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y * height))
            right_ankle = (int(landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x * width), 
                          int(landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y * height))
            left_knee = (int(landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x * width), 
                         int(landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y * height))
            right_knee = (int(landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x * width), 
                         int(landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y * height))
            left_hip = (int(landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x * width), 
                        int(landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y * height))
            right_hip = (int(landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x * width), 
                        int(landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y * height))

            # Calculate the angles
            left_shin_angle = calculate_angle(left_ankle, left_knee, left_hip)
            right_shin_angle = calculate_angle(right_ankle, right_knee, right_hip)
            left_hip_knee_angle = calculate_angle(left_hip, left_knee, left_ankle)
            right_hip_knee_angle = calculate_angle(right_hip, right_knee, right_ankle)
            left_overall_angle = calculate_angle(left_hip, left_knee, left_ankle)
            right_overall_angle = calculate_angle(right_hip, right_knee, right_ankle)

            # Display the angles on the video
            cv2.putText(image, f"Left Shin Angle: {left_shin_angle:.2f}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            cv2.putText(image, f"Right Shin Angle: {right_shin_angle:.2f}", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            cv2.putText(image, f"Left Hip-Knee Angle: {left_hip_knee_angle:.2f}", (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            cv2.putText(image, f"Right Hip-Knee Angle: {right_hip_knee_angle:.2f}", (10, 120), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            cv2.putText(image, f"Left Overall Angle: {left_overall_angle:.2f}", (10, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            cv2.putText(image, f"Right Overall Angle: {right_overall_angle:.2f}", (10, 180), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

        except:
            pass

        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(100) & 0xFF == ord('q'):
            break
        
    cap.release()
    cv2.destroyAllWindows()

def calculate_angle(a, b, c):
    a = np.array(a) 
    b = np.array(b) 
    c = np.array(c) 

    ba = a - b
    bc = c - b

    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.arccos(cosine_angle)

    return np.degrees(angle)

I0000 00:00:1703634645.646521       1 gl_context.cc:344] GL version: 2.1 (2.1 ATI-4.8.101), renderer: AMD Radeon R9 M370X OpenGL Engine


KeyboardInterrupt: 

In [10]:
len(landmarks)

for lndmrk in mp_pose.PoseLandmark:
    print(lndmrk)

PoseLandmark.NOSE
PoseLandmark.LEFT_EYE_INNER
PoseLandmark.LEFT_EYE
PoseLandmark.LEFT_EYE_OUTER
PoseLandmark.RIGHT_EYE_INNER
PoseLandmark.RIGHT_EYE
PoseLandmark.RIGHT_EYE_OUTER
PoseLandmark.LEFT_EAR
PoseLandmark.RIGHT_EAR
PoseLandmark.MOUTH_LEFT
PoseLandmark.MOUTH_RIGHT
PoseLandmark.LEFT_SHOULDER
PoseLandmark.RIGHT_SHOULDER
PoseLandmark.LEFT_ELBOW
PoseLandmark.RIGHT_ELBOW
PoseLandmark.LEFT_WRIST
PoseLandmark.RIGHT_WRIST
PoseLandmark.LEFT_PINKY
PoseLandmark.RIGHT_PINKY
PoseLandmark.LEFT_INDEX
PoseLandmark.RIGHT_INDEX
PoseLandmark.LEFT_THUMB
PoseLandmark.RIGHT_THUMB
PoseLandmark.LEFT_HIP
PoseLandmark.RIGHT_HIP
PoseLandmark.LEFT_KNEE
PoseLandmark.RIGHT_KNEE
PoseLandmark.LEFT_ANKLE
PoseLandmark.RIGHT_ANKLE
PoseLandmark.LEFT_HEEL
PoseLandmark.RIGHT_HEEL
PoseLandmark.LEFT_FOOT_INDEX
PoseLandmark.RIGHT_FOOT_INDEX


In [6]:
results.pose_landmarks

landmark {
  x: 0.6821916103363037
  y: 0.46772778034210205
  z: -1.3444254398345947
  visibility: 0.9998462796211243
}
landmark {
  x: 0.6982139348983765
  y: 0.4077970087528229
  z: -1.3035191297531128
  visibility: 0.9997185468673706
}
landmark {
  x: 0.7110324501991272
  y: 0.40778064727783203
  z: -1.3033194541931152
  visibility: 0.9996975660324097
}
landmark {
  x: 0.7231480479240417
  y: 0.40735945105552673
  z: -1.303557276725769
  visibility: 0.9996762275695801
}
landmark {
  x: 0.657467246055603
  y: 0.40302345156669617
  z: -1.3354886770248413
  visibility: 0.9997641444206238
}
landmark {
  x: 0.638907790184021
  y: 0.40033310651779175
  z: -1.3353350162506104
  visibility: 0.9997779130935669
}
landmark {
  x: 0.6190868020057678
  y: 0.3991047739982605
  z: -1.3356306552886963
  visibility: 0.9997761845588684
}
landmark {
  x: 0.726337194442749
  y: 0.4112183153629303
  z: -0.8909290432929993
  visibility: 0.9996877312660217
}
landmark {
  x: 0.5701755285263062
  y: 0.40514